In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
import pandas.io.sql as sqlio
from sqlalchemy import create_engine
import numpy as np

In [2]:
from config import password

param_dic={
    "host": "localhost",
    "database": "Energy_Analysis",
    "user" : "postgres",
    "password" : password
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
query="select * from ml_input"

ml_input_df=sqlio.read_sql_query(query,conn)

ml_input_df.head()

,sampno,gender,employment,student,education,drive_freq,transit_freq,tnc_freq,job_type,work_mode,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,190001,1,4,4,6,3,0,0,,,...,4,2,2,1,1,2,,2,2,6
1,190003,2,4,4,8,1,0,0,,,...,1,3,1,4,1,1,,2,2,5
2,190002,2,4,4,1,,0,0,,,...,2,1,,2,1,1,,2,2,3
3,190005,1,4,4,6,2,0,0,,,...,3,2,2,3,2,1,,2,2,7
4,190002,1,1,4,2,1,0,0,4,,...,2,1,,2,1,1,,2,2,3


In [11]:
# ml_input_df.select_dtypes(exclude='int64').columns

Index(['drive_freq', 'job_type', 'work_mode', 'work_distance', 'work_days',
       'school_mode', 'school_distance', 'hybrid_experience', 'past_hybrid',
       'phev_experience', 'past_phev', 'bev_experience', 'past_bev',
       'fcv_experience', 'past_fcv', 'charge_spots', 'charge_work',
       'home_parking_1', 'home_parking_2', 'home_parking_3', 'home_parking_4',
       'home_parking_5', 'home_parking_6', 'home_parking_7', 'home_parking_8',
       'home_parking_9', 'home_parking_8_x', 'home_electricity_access',
       'hsa_distance_1_1_x', 'hsa_distance_1_2_x', 'future_purchase',
       'next_purchase', 'purchase_timing', 'mode_freq_1', 'mode_freq_2',
       'mode_freq_3', 'mode_freq_4', 'mode_freq_5', 'mode_freq_6',
       'mode_freq_7', 'mode_freq_8', 'mode_freq_9', 'mode_freq_10',
       'mode_freq_11', 'mode_freq_12', 'autonomous_rideshare', 'housing_1_8_x',
       'solar_future'],
      dtype='object')

In [13]:
# ml_input_df.columns

Index(['sampno', 'gender', 'employment', 'student', 'education', 'drive_freq',
       'transit_freq', 'tnc_freq', 'job_type', 'work_mode', 'work_distance',
       'work_days', 'school_mode', 'school_distance', 'age_grp', 'california',
       'county', 'region', 'future_decision_role', 'num_hh_vehicles',
       'household_members_1', 'household_members_2', 'household_members_3',
       'household_members_4', 'tot_hh_members', 'hybrid_experience',
       'past_hybrid', 'phev_experience', 'past_phev', 'bev_experience',
       'past_bev', 'fcv_experience', 'past_fcv', 'charge_spots', 'charge_work',
       'home_parking_1', 'home_parking_2', 'home_parking_3', 'home_parking_4',
       'home_parking_5', 'home_parking_6', 'home_parking_7', 'home_parking_8',
       'home_parking_9', 'home_parking_8_x', 'home_electricity_access',
       'hydrogen_station_awareness', 'hsa_distance_1_1_x',
       'hsa_distance_1_2_x', 'future_purchase', 'next_purchase',
       'purchase_timing', 'modes_used_1', 'm

## Clean Data

## Split Data into Training and Testing

In [ ]:
X=ml_input.drop(columns='ev_flag', axis='columns')
y=ml_input['ev_flag']

X.describe()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(random_state=1)
brf.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
dict(zip(X.columns,sorted(importances,reverse=True)))